# import modules

In [1]:
from datetime import datetime
import gzip
import numpy as np
from scipy.linalg import eig,norm
from math import *
from qutip import parfor,qeye,sigmax,sigmay,sigmaz
from qutip import *
np.set_printoptions(precision=3)
from IPython.display import display
import seaborn as sns
import mpltex
import matplotlib.pyplot as plt
try:    
    import cPickle as pickle 
except:    
    import pickle as pickle

%matplotlib inline

import pandas as pd
from os import listdir
from os.path import isfile, join

用于plot的数据主要来自./data/6.20(6.22\6.24\6.25\6.27\bp_8.7) 等几个文件夹，并且用文件 AnalyzeData.ipynb 进行数据分析，得到下面的结果:
 - 每个数据点由100次测量统计而成，100个数据每20一组分为5组，统计平均值和标准差
 - 每次测量N=5，readout=100

## import data

固定$\alpha=200e6$，变化tau的结果,tau=10**(1,0.5,0,-0.5,-1,-1.5,-2,-2.5,-3,-3.5,-4)

In [27]:
scheme1_se_tau = [0.982,0.981,0.980,0.972,0.950,0.887,0.766,0.581,0.442,0.473,0.439]
scheme1_cpmg_tau = [0.997,0.998,0.997,0.994,0.988,0.9676,0.920,0.785,0.626,0.523,0.493]
scheme2_tau = [0.948,0.939,0.934,0.937,0.922,0.889,0.761,0.540,0.305,0.224,0.238]

scheme1_cpmg_tau_exper = [[0.88, 0.89, 0.88, 0.90, 0.91],[0.89, 0.91, 0.90, 0.90, 0.90],[0.88, 0.89, 0.90, 0.90, 0.88],\
                    [0.88, 0.88, 0.89, 0.88, 0.9],[0.82, 0.88, 0.9, 0.89, 0.87],[0.84, 0.82, 0.83, 0.86, 0.89],\
                    [0.83, 0.83, 0.77, 0.75, 0.83],[0.68, 0.77, 0.56, 0.71, 0.7],[0.35, 0.49, 0.32, 0.62, 0.56],\
                    [0.46, 0.46, 0.49, 0.54, 0.58],[0.12, 0.39, 0.53, 0.33, 0.47]]
scheme1_se_tau_exper = [[0.89, 0.86, 0.9, 0.86, 0.83],[0.86, 0.87, 0.86, 0.87, 0.87],[0.85, 0.88, 0.84, 0.85, 0.86],\
                    [0.85, 0.84, 0.81, 0.86, 0.85],[0.84, 0.83, 0.87, 0.85, 0.86],[0.78, 0.76, 0.76, 0.71, 0.79],\
                    [0.73, 0.6, 0.72, 0.56, 0.78],[0.42, 0.47, 0.65, 0.39, 0.48],[0.29, 0.19, 0.6, 0.24, 0.35],\
                    [0.55, 0.6, 0.56, 0.31, 0.11],[0.53, 0.52, 0.43, 0.53, 0.23]]
scheme2_tau_exper = [[0.7, 0.75, 0.79, 0.74, 0.76],[0.82, 0.8, 0.83, 0.75, 0.81],[0.83, 0.77, 0.75, 0.78, 0.74],\
              [0.79, 0.83, 0.79, 0.75, 0.78],[0.81, 0.76, 0.76, 0.75, 0.82],[0.6, 0.74, 0.77, 0.61, 0.79],\
              [0.6, 0.71, 0.69, 0.62, 0.75],[0.44, 0.56, 0.44, 0.47, 0.28],[0.43, 0.5, 0.37, 0.29, 0.45],\
              [0.21, 0.48, 0.4, 0.21, 0.07],[0.41, 0.21, 0.04, 0.24, 0.05]]

固定$\tau=0.01$,变化$\alpha$的结果，alpha=100e(4,4.5,5,5.5,6,6.5,7,7.5)

In [3]:
scheme1_se_alpha = [0.98,0.98,0.98,0.97,0.878,0.560,0.412,0.270]
scheme1_cpmg_alpha = [0.999,0.999,0.998,0.993,0.970,0.839,0.392,0.202]
scheme2_alpha = [0.997,0.994,0.988,0.967,0.918,0.666,0.300,0.091]

FID

In [4]:
FID = [[0.35, 0.37, 0.37, 0.22, 0.05],[0.67, 0.46, 0.29, 0.3, 0.71],[0.61, 0.58, 0.61, 0.67, 0.33],\
 [0.87, 0.86, 0.84, 0.9, 0.87], [0.98, 0.98, 0.98, 0.98, 0.97],[0.92, 0.96, 0.93, 0.93, 0.9],\
 [0.71, 0.73, 0.79, 0.7, 0.72],[0.52, 0.47, 0.49, 0.59, 0.52],[0.43, 0.51, 0.53, 0.1, 0.1]]

不添加噪声的情况下

In [5]:
bp_se_no_noise = [[1.0], [0.96], [0.95], [0.81], [0.98], [0.91], [0.88], [0.4], [0.9]]
bp_cpmg_no_noise = [[0.98], [0.98], [0.95], [0.69], [0.92], [0.98], [0.92], [0.58], [0.84]]
scheme1_se_no_noise = [[1.03], [0.97], [0.81], [0.97], [0.71], [0.81], [0.38], [0.51], [0.89]]
scheme1_cpmg_no_noise = [[0.99], [0.91], [0.73], [0.34], [0.89], [1.0], [0.72], [0.43], [0.98]]
scheme2_no_noise = [[0.98], [0.96], [0.99], [0.87], [0.97], [0.98], [0.97], [0.45], [0.88]]

理论模拟出来的值（用统计400次平均的结果）

In [6]:
bp_se_thero = [0.99,0.99,0.97,0.92,0.78,0.62,0.45,0.31,0.20]
scheme1_se_thero = [0.99,0.99,0.99,0.98,0.98,0.98,0.98,0.96,0.99]
scheme2_thero = [0.99,0.99,0.99,0.99,0.98,0.95,0.92,0.95,0.97]
bp_cpmg_thero = [0.99,0.99,0.97,0.89,0.78,0.62,0.53,0.26,0.19]
scheme1_cpmg_thero = [0.99,0.99,0.99,0.98,0.98,0.99,0.98,0.95,0.99]

实验测出的值，已经经过处理

In [7]:
scheme1_se_exper = [[0.977,0.966,0.969,0.973,0.976],[0.978,0.971,0.975,0.965,0.986],[0.962,0.919,0.948,0.961,0.958],\
                    [0.9,0.889,0.889,0.907,0.89],[0.859,0.855,0.854,0.877,0.878],[0.928,0.906,0.914,0.924,0.916],\
                    [0.953,0.955,0.935,0.944,0.941],[0.8,0.818,0.828,0.794,0.829],[0.889,0.894,0.892,0.878,0.890]]
se_exper = [[0.967,0.975,0.972,0.976,0.981],[0.981,0.968,0.971,0.981,0.988],[0.951,0.943,0.948,0.954,0.930],\
            [0.844,0.850,0.795,0.876,0.824],[0.716,0.636,0.554,0.627,0.690],[0.46,0.340,0.688,0.634,0.619],\
            [0.366,0.479,0.513,0.655,0.144],[0.266,0.435,0.488,0.057,0.064],[0.129,0.081,0.385,0.242,0.054]]
scheme2_exper = [[0.951,0.952,0.957,0.955,0.960],[0.932,0.939,0.962,0.945,0.933],[0.953,0.967,0.961,0.964,0.959],\
           [0.983,0.974,0.986,0.977,0.981],[0.935,0.931,0.931,0.936,0.929],[0.838,0.786,0.847,0.846,0.829],\
           [0.802,0.821,0.796,0.755,0.758],[0.917,0.940,0.899,0.896,0.870],[0.877,0.926,0.883,0.901,0.880]]
cpmg_exper = [[0.977,0.977,0.971,0.982,0.974],[0.961,0.975,0.971,0.964,0.989],[0.933,0.960,0.961,0.969,0.974],\
              [0.877,0.847,0.828,0.830,0.877],[0.705,0.672,0.658,0.730,0.630],[0.469,0.669,0.672,0.648,0.621],\
              [0.535,0.364,0.420,0.363,0.535],[0.471,0.410,0.691,0.495,0.074],[0.433,0.126,0.295,0.383,0.351]]
scheme1_cpmg_exper = [[0.974,0.984,0.977,0.979,0.987],[0.980,0.970,0.736,0.978,0.968],[0.968,0.964,0.967,0.968,0.967],\
                [0.924,0.922,0.918,0.926,0.918],[0.866,0.855,0.853,0.862,0.853],[0.940,0.938,0.939,0.950,0.941],\
                [0.939,0.958,0.961,0.962,0.956],[0.853,0.762,0.834,0.780,0.762],[0.883,0.914,0.927,0.887,0.901]]

不加噪声时，各个方案的结果

In [8]:
cpmg_no_noise = [0.981,0.848,0.831,0.812,0.948,0.979,0.959,0.936,0.999,0.934]
se_no_noise =[0.985,0.96,1,1,1,0.987,0.932,0.926,0.995,0.980]
scheme1_cp_no_noise = [0.994,0.961,0.903,0.988,1,1,0.993,0.916,0.988,1]
scheme2_no_noise = [0.974,0.953,0.92,0.978,0.975,0.782,0.814,0.965,0.973,1]
scheme1_se_no_noise = [0.98,0.993,0.97,0.99,0.972,0.948,0.944,0.984,0.929,0.945]

Berry phase 的 结果

In [9]:
bp_se = [[0.2, 0.19, 0.18, 0.2, 0.2],[0.5, -1.44, 0.43, 0.37, -2.03],[1.38, 1.42, 1.38, 1.39, 1.44],\
 [2.39, 2.54, 2.41, 2.28, 2.69],[-2.6, -2.73, -2.17, -2.44, -2.6],[-0.44, -0.66, -1.12, -1.07, -0.77],\
 [0.56, 0.28, -0.14, 0.64, 0.3],[1.53, 2.16, 2.02, 2.28, -2.65],[0.94, 1.69, 1.65, -2.73, 0.11]]
bp_cpmg = [[0.14, 0.18, 0.19, 0.13, 0.16],[0.62, 0.63, 0.59, 0.58, 0.62],[1.47, 1.39, 1.41, 1.27, 1.3],\
 [2.35, 2.44, 2.4, 2.41, 2.35],[-2.55, -2.76, -2.4, -2.39, -3.01],[-1.01, -1.1, -0.79, -1.03, -1.24],\
 [0.81, 1.01, 0.1, 1.32, -0.46],[2.49, 2.04, 1.5, 1.17, 1.43],[-2.34, 3.11, 1.15, -1.52, -2.09]]
bp_scheme1_cpmg = [[0.13, 0.17, 0.17, 0.17, 0.17],[0.69, 0.69, 0.7, 0.7, 0.7],[1.5, 1.53, 1.54, 1.53, 1.49],\
 [2.66, 2.67, 2.61, 2.63, 2.66],[-2.06, -2.04, -2.09, -2.08, -2.06],[-0.35, -0.37, -0.34, -0.33, -0.38],\
 [1.41, 1.41, 1.44, 1.4, 1.41],[-2.79, -2.75, -2.8, -2.79, -2.82],[-0.59, -0.65, -0.66, -0.67, -0.65]]
bp_scheme1_se = [[0.24, 0.23, 0.22, 0.21, 0.22],[0.73, 0.72, 0.74, 0.72, 0.75],[1.66, 1.56, 1.57, 1.52, 1.56],\
 [2.7, 2.71, 2.69, 2.71, 2.73],[-2.04, -2.06, -2.08, -2.06, -2.04],[-0.34, -0.31, -0.3, -0.27, -0.29],\
 [1.46, 1.43, 1.46, 1.45, 1.48],[-2.81, -2.75, -2.82, -2.77, -2.82],[-0.62, -0.64, -0.62, -0.62, -0.62]]
bp_scheme2 = [[2.98, 2.98, 2.99, 2.95, 2.99],[2.49, 2.5, 2.5, 2.52, 2.52],[1.74, 1.73, 1.7, 1.74, 1.71],\
 [0.71, 0.71, 0.7, 0.69, 0.7],[-0.49, -0.5, -0.48, -0.53, -0.52],[-2.01, -2.12, -2.04, -2.08, -2.01],\
 [2.37, 2.32, 2.38, 2.32, 2.38],[0.57, 0.62, 0.63, 0.53, 0.46],[-1.17, -1.12, -1.16, -1.14, -1.19]]
bp = -np.cos(np.array([0.16,0.31,0.47,0.63,0.79,0.94,1.1,1.26,1.41])) * 4 * np.pi

### Fig8

In [10]:
fig_8 = pd.DataFrame(columns=['bp_scheme1_se','bp_scheme1_se_std','bp_scheme1_cpmg','bp_scheme1_cpmg_std','bp_scheme2','bp_scheme2_std','bp'])
fig_8.loc[:,'bp_scheme1_se'] = np.array(bp_scheme1_se).mean(axis=1)
fig_8.loc[:,'bp_scheme1_se_std'] = np.array(bp_scheme1_se).std(axis=1)
fig_8.loc[0:3,'bp_scheme1_se'] = fig_8.loc[0:3,'bp_scheme1_se'] - 4*np.pi
fig_8.loc[4:6,'bp_scheme1_se'] = fig_8.loc[4:6,'bp_scheme1_se'] - 2*np.pi
fig_8.loc[:,'bp_scheme1_cpmg'] = np.array(bp_scheme1_cpmg).mean(axis=1)
fig_8.loc[:,'bp_scheme1_cpmg_std'] = np.array(bp_scheme1_cpmg).std(axis=1)
fig_8.loc[0:3,'bp_scheme1_cpmg'] = fig_8.loc[0:3,'bp_scheme1_cpmg'] - 4*np.pi
fig_8.loc[4:6,'bp_scheme1_cpmg'] = fig_8.loc[4:6,'bp_scheme1_cpmg'] - 2*np.pi
fig_8.loc[:,'bp_scheme2'] = np.array(bp_scheme2).mean(axis=1)
fig_8.loc[:,'bp_scheme2_std'] = np.array(bp_scheme2).std(axis=1)
fig_8.loc[0:5,'bp_scheme2'] = fig_8.loc[0:5,'bp_scheme2']*(-1) - 3*np.pi
fig_8.loc[6:7,'bp_scheme2'] = fig_8.loc[6:7,'bp_scheme2']*(-1) - 1*np.pi
fig_8.loc[:,'bp'] = bp
fig_8 = fig_8 / np.pi
fig_8

,bp_scheme1_se,bp_scheme1_se_std,bp_scheme1_cpmg,bp_scheme1_cpmg_std,bp_scheme2,bp_scheme2_std,bp
0,-3.928699,0.003246,-3.948434,0.005093,-3.947927,0.004678,-3.948909
1,-3.766997,0.003712,-3.778456,0.001559,-3.797685,0.003820,-3.809334
2,-3.498980,0.014739,-3.516806,0.006172,-3.548766,0.005172,-3.566273
3,-3.138017,0.004223,-3.157752,0.007146,-3.223454,0.002382,-3.232110
4,-2.654445,0.004764,-2.657628,0.005550,-2.839572,0.005904,-2.815381
5,-2.096130,0.007369,-2.112682,0.005904,-2.346828,0.013565,-2.359152
6,-1.536541,0.005172,-1.549910,0.004318,-1.749301,0.008913,-1.814384
7,-0.889358,0.009137,-0.888085,0.007259,-1.178890,0.019868,-1.223268
8,-0.198625,0.002546,-0.204992,0.008913,-0.367966,0.007692,-0.640417


In [11]:
fig_8.to_csv('fig_8.csv')

### Fig7

In [12]:
fig_7 = pd.DataFrame(columns=['bp_se','bp_se_std','bp_cpmg','bp_cpmg_std','bp'])
fig_7.loc[:,'bp_se'] = np.array(bp_se).mean(axis=1)
fig_7.loc[:,'bp_se_std'] = np.array(bp_se).std(axis=1)
fig_7.loc[0:3,'bp_se'] = fig_7.loc[0:3,'bp_se'] - 4*np.pi
fig_7.loc[4:7,'bp_se'] = fig_7.loc[4:7,'bp_se'] - 2*np.pi
fig_7.loc[:,'bp_cpmg'] = np.array(bp_cpmg).mean(axis=1)
fig_7.loc[:,'bp_cpmg_std'] = np.array(bp_cpmg).std(axis=1)
fig_7.loc[0:3,'bp_cpmg'] = fig_7.loc[0:3,'bp_cpmg'] - 4*np.pi
fig_7.loc[4:7,'bp_cpmg'] = fig_7.loc[4:7,'bp_cpmg'] - 2*np.pi
fig_7.loc[:,'bp'] = bp
fig_7 = fig_7 / np.pi
fig_7

,bp_se,bp_se_std,bp_cpmg,bp_cpmg_std,bp
0,-3.938248,0.002546,-3.949070,0.007259,-3.948909
1,-4.138146,0.343554,-3.806468,0.006172,-3.809334
2,-3.553730,0.007639,-3.564552,0.023339,-3.566273
3,-3.216321,0.044808,-3.239239,0.011209,-3.232110
4,-2.798321,0.061241,-2.834609,0.075040,-2.815381
5,-2.258468,0.081113,-2.329132,0.046547,-2.359152
6,-1.895594,0.086973,-1.823020,0.206019,-1.814384
7,-1.660045,0.597282,-1.450597,0.151331,-1.223268
8,0.105679,0.520630,-0.107589,0.676399,-0.640417


In [13]:
fig_7.to_csv('fig_7.csv')

## Export data

### Fig1

In [23]:
fig_1 = pd.DataFrame(columns=['se_exper','se_std','cpmg_exper','cpmg_std','bp_se_thero','bp_cpmg_thero'])
fig_1.loc[:,'se_exper'] = np.array(se_exper).mean(axis=1) - np.concatenate(np.array(bp_se_no_noise)) + 1
fig_1.loc[:,'se_std'] = np.array(se_exper).std(axis=1)
fig_1.loc[:,'cpmg_exper'] = np.array(cpmg_exper).mean(axis=1) - np.concatenate(np.array(bp_cpmg_no_noise)) + 1
fig_1.loc[:,'cpmg_std'] = np.array(cpmg_exper).std(axis=1)
fig_1.loc[:,'bp_se_thero'] = bp_se_thero
fig_1.loc[:,'bp_cpmg_thero'] = bp_cpmg_thero
fig_1

,se_exper,se_std,cpmg_exper,cpmg_std,bp_se_thero,bp_cpmg_thero
0,0.9742,0.004622,0.9962,0.003655,0.99,0.99
1,1.0178,0.007305,0.9920,0.009839,0.99,0.99
2,0.9952,0.008424,1.0094,0.014179,0.97,0.97
3,1.0278,0.027088,1.1618,0.021609,0.92,0.89
4,0.6646,0.056127,0.7590,0.035123,0.78,0.78
5,0.6382,0.128876,0.6358,0.075634,0.62,0.62
6,0.5514,0.170761,0.5234,0.077585,0.45,0.53
7,0.8620,0.180139,0.8482,0.200573,0.31,0.26
8,0.2782,0.121763,0.4776,0.105765,0.20,0.19


In [156]:
fig_1.to_csv('fig_1.csv')

### Fig2

In [25]:
fig_2 = pd.DataFrame(columns=['scheme1_se_exper','scheme1_se_std','scheme1_cpmg_exper','scheme1_cpmg_std',\
                              'scheme2_exper','scheme2_std','scheme1_se_thero','scheme1_cpmg_thero', 'scheme2_thero'])
fig_2.loc[:,'scheme1_se_exper'] = np.array(scheme1_se_exper).mean(axis=1) 
fig_2.loc[:,'scheme1_se_std'] = np.array(scheme1_se_exper).std(axis=1)
fig_2.loc[:,'scheme2_exper'] = np.array(scheme2_exper).mean(axis=1)
fig_2.loc[:,'scheme2_std'] = np.array(scheme2_exper).std(axis=1)
fig_2.loc[:,'scheme1_cpmg_exper'] = np.array(scheme1_cpmg_exper).mean(axis=1)
fig_2.loc[:,'scheme1_cpmg_std'] = np.array(scheme1_cpmg_exper).std(axis=1)
fig_2.loc[:,'scheme1_se_thero'] = scheme1_se_thero
fig_2.loc[:,'scheme2_thero'] = scheme2_thero
fig_2.loc[:,'scheme1_cpmg_thero'] = scheme1_cpmg_thero
fig_2

,scheme1_se_exper,scheme1_se_std,scheme1_cpmg_exper,scheme1_cpmg_std,scheme2_exper,scheme2_std,scheme1_se_thero,scheme1_cpmg_thero,scheme2_thero
0,0.9722,0.004167,0.9802,0.004707,0.9550,0.003286,0.99,0.99,0.99
1,0.9750,0.007014,0.9264,0.095309,0.9422,0.010943,0.99,0.99,0.99
2,0.9496,0.016082,0.9668,0.001470,0.9608,0.004750,0.99,0.99,0.99
3,0.8950,0.007294,0.9216,0.003200,0.9802,0.004261,0.98,0.98,0.99
4,0.8646,0.010670,0.8578,0.005269,0.9324,0.002653,0.98,0.98,0.98
5,0.9176,0.007736,0.9416,0.004317,0.8292,0.022551,0.98,0.99,0.95
6,0.9456,0.007473,0.9552,0.008376,0.7864,0.025788,0.98,0.98,0.92
7,0.8138,0.014372,0.7982,0.038044,0.9044,0.023277,0.96,0.95,0.95
8,0.8886,0.005571,0.9024,0.016463,0.8934,0.018315,0.99,0.99,0.97


In [165]:
fig_2.to_csv('fig_2.csv')

### Fig3

In [28]:
fig_3 = pd.DataFrame(columns=['scheme1_se_tau_exper','scheme1_se_tau_std','scheme1_cpmg_tau_exper','scheme1_cpmg_tau_std',\
                              'scheme2_tau_exper','scheme2_tau_std','scheme1_se_tau','scheme1_cpmg_tau','scheme2_tau'])
fig_3.loc[:,'scheme1_se_tau_exper'] = np.array(scheme1_se_tau_exper).mean(axis=1)
fig_3.loc[:,'scheme1_se_tau_std'] = np.array(scheme1_se_tau_exper).std(axis=1)
fig_3.loc[:,'scheme1_cpmg_tau_exper'] = np.array(scheme1_cpmg_tau_exper).mean(axis=1)
fig_3.loc[:,'scheme1_cpmg_tau_std'] = np.array(scheme1_cpmg_tau_exper).std(axis=1)
fig_3.loc[:,'scheme2_tau_exper'] = np.array(scheme2_tau_exper).mean(axis=1)
fig_3.loc[:,'scheme2_tau_std'] = np.array(scheme2_tau_exper).std(axis=1)
fig_3.loc[:,'scheme1_se_tau'] = scheme1_se_tau
fig_3.loc[:,'scheme1_cpmg_tau'] = scheme1_cpmg_tau
fig_3.loc[:,'scheme2_tau'] = scheme2_tau
fig_3

,scheme1_se_tau_exper,scheme1_se_tau_std,scheme1_cpmg_tau_exper,scheme1_cpmg_tau_std,scheme2_tau_exper,scheme2_tau_std,scheme1_se_tau,scheme1_cpmg_tau,scheme2_tau
0,0.868,0.024819,0.892,0.011662,0.748,0.029257,0.982,0.9970,0.948
1,0.866,0.004899,0.900,0.006325,0.802,0.027857,0.981,0.9980,0.939
2,0.856,0.013565,0.890,0.008944,0.774,0.031369,0.980,0.9970,0.934
3,0.842,0.017205,0.886,0.008000,0.788,0.025612,0.972,0.9940,0.937
4,0.850,0.014142,0.872,0.027857,0.780,0.028983,0.950,0.9880,0.922
5,0.760,0.027568,0.848,0.024819,0.702,0.080846,0.887,0.9676,0.889
6,0.678,0.083522,0.802,0.034871,0.674,0.056071,0.766,0.9200,0.761
7,0.482,0.090200,0.684,0.068877,0.438,0.090421,0.581,0.7850,0.540
8,0.334,0.143192,0.468,0.116516,0.408,0.072222,0.442,0.6260,0.305
9,0.426,0.188106,0.506,0.047159,0.274,0.147051,0.473,0.5230,0.224


In [29]:
fig_3.to_csv('fig_3.csv')

In [626]:
rho_aver = 0
P = len(Rho)
for j in Rho[:P]:
    rho_aver += j
rho_aver = rho_aver / P
print(abs(rho_aver[1][0][0])/0.5)

0.8293795807280101


In [ ]:
[0.859,0.855,0.854,0.877,0.878],[0.8,0.818,0.828,0.794,0.829]   # scheme1_se  0.79 和 1.26 两个点（dt=1e-9）

In [ ]:
[[0.977,0.966,0.969,0.973,0.976],[0.978,0.971,0.975,0.965,0.986],[0.962,0.919,0.948,0.961,0.958],
 [0.9,0.889,0.889,0.907,0.89],[0.849,0.867,0.826,0.835,0.835],[0.928,0.906,0.914,0.924,0.916],
 [0.953,0.955,0.935,0.944,0.941],[0.78,0.755,0.761,0.731,0.789],[0.889,0.894,0.892,0.878,0.890]]  # scheme1_se

 [[0.967,0.975,0.972,0.976,0.981],[0.981,0.968,0.971,0.981,0.988],[0.951,0.943,0.948,0.954,0.930],
 [0.844,0.850,0.795,0.876,0.824],[0.716,0.636,0.554,0.627,0.690],[0.46,0.340,0.688,0.634,0.619],
 [0.366,0.479,0.513,0.655,0.144],[0.266,0.435,0.488,0.057,0.064],[0.129,0.081,0.385,0.242,0.054]] # measuer_bp_se
    
[[0.951,0.952,0.957,0.955,0.960],[0.932,0.939,0.962,0.945,0.933],[0.953,0.967,0.961,0.964,0.959],
 [0.983,0.974,0.986,0.977,0.981],[0.935,0.931,0.931,0.936,0.929],[0.838,0.786,0.847,0.846,0.829],
 [0.802,0.821,0.796,0.755,0.758],[0.917,0.940,0.899,0.896,0.870],[0.877,0.926,0.883,0.901,0.880]]  # scheme2

[[0.977,0.977,0.971,0.982,0.974],[0.961,0.975,0.971,0.964,0.989],[0.933,0.960,0.961,0.969,0.974],
 [0.877,0.847,0.828,0.830,0.877],[0.705,0.672,0.658,0.730,0.630],[0.469,0.669,0.672,0.648,0.621],
 [0.535,0.364,0.420,0.363,0.535],[0.471,0.410,0.691,0.495,0.074],[0.433,0.126,0.295,0.383,0.351]]  # measure_bp_cpmg

[[0.974,0.984,0.977,0.979,0.987],[0.980,0.970,0.736,0.978,0.968],[0.968,0.964,0.967,0.968,0.967],
 [0.924,0.922,0.918,0.926,0.918],[0.866,0.855,0.853,0.862,0.853],[0.940,0.938,0.939,0.950,0.941],
 [0.939,0.958,0.961,0.962,0.956],[0.853,0.762,0.834,0.780,0.762],[0.883,0.914,0.927,0.887,0.901]]  # scheme1_cpmg

## 极大似然估计

In [30]:
# 2017-11-10 修正正定问题
import qutip as qu
import qinfer as qi 


def ML_EstimateState(n,N,MeasurBasis):
    ''' Give result and measurement Basis to estimate the density matrix of the state
    
    n: success measurement shots
    N: total measurement shots
    MeasurBasis: measurement basis used in the experiment
    '''
    # judge how many qubits
    dims = MeasurBasis[0].dims[0]
    dim = np.product(dims)
    
    def Gen_GM_Basis (dims): 
        ibasis = qi.tomography.gell_mann_basis(dims[0])
        ibasis = ibasis[range(len(ibasis))]
        if (len(dims) > 1):
            return [ qu.tensor(e, e1) for e in ibasis for e1 in  Gen_GM_Basis(dims[1:])]
        else:
            return ibasis
        
    B_all = Gen_GM_Basis(dims)
    B0 = B_all[0]   #  1/d matrix, d is the dimension = 2**N
    B  = B_all[1:]  # pauli matrix list, length d-1
    # generate tranform matrix X
    X  = [[(qu.ket2dm(Pi)*Bj).tr() for Bj in B] for Pi in MeasurBasis]
    X  = np.array(X)
    
    f = (n+0.5)/(N+1)
    Y = f- 1.0/dim
    
    a = np.sqrt(N/(f*(1-f)))
    Y1 = a*Y
    X1 = np.array([np.array(Xk)*ak for Xk,ak in zip(X,a)])
    
    # calculate initial value by linear transform
    #x00 = np.real((np.linalg.inv((X.T.dot(X))).dot(X.T)).dot(Y))
    x00 = np.zeros(dim**2-1)
    
    from scipy.optimize import minimize
    # estimate bound
    #bound = [(0.0, None) for x in x00]

    # estimate constraints
    def con_fun(x):
        d = np.sqrt(len(x)+1)
        return 1.0-1.0/d - np.linalg.norm(x, ord=2)**2
    constrain = {
        'type': 'ineq',
        'fun': con_fun
    }
    # estimate function
    def estimate_fun(theta):
        return (np.linalg.norm(Y1-X1.dot(theta),ord=2))
    # to estimate values
#     res = minimize(estimate_fun,x00,constraints=constrain, method = 'SLSQP')
    res = minimize(estimate_fun,x00,constraints=constrain, method = 'COBYLA')
    rho = np.sum([x*Bi for x, Bi in zip(res.x,B)],axis=0) + B0*B0
    return (rho,res)

In [31]:
import numpy as np
from qutip import basis,fidelity
s0=basis(2,0)
s1=basis(2,1)
s2=basis(2,0)+basis(2,1)
s3=basis(2,0)+1.0j*basis(2,1)
s4=basis(2,0)-basis(2,1)
s5=basis(2,0)-1.0j*basis(2,1)

s2 /= s2.norm()
s3 /= s3.norm()
s4 /= s4.norm()
s5 /= s5.norm()


proj=[s0,s2,s3]
y = np.array([Z,X,Y])
N =10000*np.ones(3)

# proj=[s0,s1,s3,s2,s5,s4]
# y = np.array([0.45,  0.49,  0.97,  0.51,  0.00,  0.51]) # X state
# N =10000*np.ones(6)

# proj=[s0,s1,s3,s2]
# y = np.array([0.45,  0.49,  0.97,  0.51])
# N =10000*np.ones(4)

# proj=[s0,s1,s5,s4]
# y = np.array([0.45,  0.49,  0,  0.51])
# N =10000*np.ones(4)

a,b = ML_EstimateState(y*10000,N,proj)

print(df.iloc[0].values[3:-2])
print(b.x)
# print((a*a).tr())
a.eigenenergies()
b

[ 0.    0.49]
[ 0.037 -0.7   -0.016]


     fun: 0.30636206443576858
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 113
  status: 1
 success: True
       x: array([ 0.037, -0.7  , -0.016])

In [32]:
a

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.526+0.j    -0.495+0.012j]
 [-0.495-0.012j  0.474+0.j   ]]